In [1]:
import os
import numpy as np
import pandas as pd 
import math 

from os import walk
import tensorflow.compat.v1 as tf

from sklearn.model_selection import train_test_split

In [2]:
labels_path = 'labels.csv'
images_path = '../completed/'
IMG_SIZE = 64
IMG_DEPTH = 32

In [3]:
candidates = pd.read_csv(labels_path,index_col=0)
candidates.head()

,seriesuid,label
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,1
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100332161840...,0
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,1
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100530488926...,0
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100620385482...,0


In [4]:
def load_slices(file):
    return np.load(images_path+file+".npy")

In [62]:
data = []
for index, candidat in candidates.iterrows():
    slices = load_slices(candidat.seriesuid)
    label = []
    if candidat.label == 1:
        label = [1,0]
    else:
        label = [0,1]
        
    data.append([slices,label])


In [63]:
data = data[-400:]
print(len(data))

400


In [65]:
Xtrain,Xtest,Ytrain,Ytest= train_test_split([i[0] for i in data],[i[1] for i in data],train_size=0.5,random_state=1001)
print(np.sum(Ytrain,axis=0))
print(np.sum(Ytest,axis=0))

[139  61]
[130  70]


In [60]:
 for xD,yD in zip(Xtrain,Ytrain):
    print(xD,yD)

[[[ -994.69230769 -1009.30769231  -979.53846154 ...  -996.61538462
    -992.46153846  -996.69230769]
  [ -985.07692308  -998.84615385  -999.92307692 ...  -999.53846154
    -995.46153846 -1007.46153846]
  [ -987.92307692  -992.46153846 -1007.23076923 ...  -984.61538462
    -998.92307692  -988.38461538]
  ...
  [ -893.53846154  -902.23076923  -906.15384615 ...  -908.23076923
    -892.15384615  -905.76923077]
  [ -903.          -897.15384615  -897.07692308 ...  -892.15384615
    -901.23076923  -916.30769231]
  [ -893.46153846  -901.46153846  -900.38461538 ...  -906.53846154
    -885.07692308  -897.        ]]

 [[-1006.84615385  -994.23076923  -996.61538462 ...  -994.
   -1010.07692308 -1009.23076923]
  [-1006.23076923 -1011.61538462 -1007.07692308 ... -1000.23076923
    -997.46153846  -994.30769231]
  [-1013.76923077 -1002.23076923  -990.84615385 ...  -993.46153846
    -998.          -998.92307692]
  ...
  [ -880.46153846  -898.23076923  -899.61538462 ...  -884.38461538
    -904.69230769 

In [ ]:
n_classes = 2
batch_size = 10
tf.compat.v1.disable_eager_execution()

x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

keep_rate = 0.8
rate = 1 - tf.placeholder(tf.float32)

def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1,1,1,1,1], padding='SAME')

def maxpool3d(x):
    #                        size of window         movement of window
    return tf.nn.max_pool3d(x, ksize=[1,2,2,2,1], strides=[1,2,2,2,1], padding='SAME')



def convolutional_neural_network(x):
    weights = {'W_conv1':tf.Variable(tf.random_normal([5,5,5,1,32])),
               'W_conv2':tf.Variable(tf.random_normal([5,5,5,32,64])),
               'W_fc':tf.Variable(tf.random_normal([IMG_SIZE*IMG_SIZE*IMG_DEPTH,1024])),
               'out':tf.Variable(tf.random_normal([1024, n_classes]))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
               'b_conv2':tf.Variable(tf.random_normal([64])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([n_classes]))}

    x = tf.reshape(x, shape=[-1, IMG_SIZE, IMG_SIZE, IMG_DEPTH,1])
    print(x)
    conv1 = tf.nn.relu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool3d(conv1)
    
    conv2 = tf.nn.relu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool3d(conv2)

    fc = tf.reshape(conv2,[-1, IMG_SIZE*IMG_SIZE*IMG_DEPTH])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)

    output = tf.matmul(fc, weights['out'])+biases['out']

    return output

def train_neural_network(x):
    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(prediction,y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    saver = tf.train.Saver()

    hm_epochs = 3
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            
            for xD,yD in zip(Xtrain,Ytrain):
                _, c = sess.run([optimizer, cost], feed_dict={x: xD, y: yD})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)
            correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
            print('Accuracy:',accuracy.eval({x: Xtest, y:Ytest}))
            #save_path = saver.save(sess, "/model/checkpoint.ckpt")
           # print(save_path)
       
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        print(correct)
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x: Xtest, y:Ytest}))
              
train_neural_network(x)

Tensor("Reshape_22:0", shape=(None, 64, 64, 32, 1), dtype=float32)
Epoch 0 completed out of 3 loss: -2.633737687749299e+16
Accuracy: 0.65
